In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt


### **Predicting Concrete Compressive Strength (MPa)**

Build an Artificial Neural Network (ANN) to accurately predict the compressive strength of concrete (measured in MPa) based on the given mixture composition and the specific age of the concrete (in days) using the provided dataset.


In [ ]:
df = pd.read_csv('concrete_data.csv')
df.drop_duplicates(inplace=True)


X = df.drop(columns=['Strength'])  
y = df['Strength']              
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = Sequential(name="Concrete_Strength_ANN")
model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1])) 
model.add(Dense(32, activation='relu')) 
model.add(Dense(32, activation='relu')) 
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train_scaled, y_train, epochs=200, batch_size=32, validation_split=0.2, verbose=1)
loss, mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")

y_pred = model.predict(X_test_scaled)
y_pred = model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")


plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.title('Training and Validation MAE')
plt.show()


# hyperparameter tunning

In [ ]:
from  tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_model(learning_rate, dropout_rate, activation_function, init, neuron1, neuron2):
    model = Sequential()
    model.add(Dense(neuron1, input_dim=X_train_scaled.shape[1], kernel_initializer=init, activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2, kernel_initializer=init, activation=activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))  # Output layer for regression

    adam = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=adam, metrics=['mae'])
    return model

model = KerasRegressor(build_fn=create_model, verbose=0)

batch_size = [16, 32, 64]
epochs = [100, 200]
learning_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.2, 0.3, 0.5]
activation_function = ['relu', 'tanh', 'linear']
init = ['uniform', 'normal']
neuron1 = [32, 64, 128]
neuron2 = [16, 32, 64]

param_grids = dict(
    batch_size=batch_size,
    epochs=epochs,
    learning_rate=learning_rate,
    dropout_rate=dropout_rate,
    activation_function=activation_function,
    init=init,
    neuron1=neuron1,
    neuron2=neuron2
)

grid = GridSearchCV(estimator=model, param_grid=param_grids, cv=KFold(), verbose=10)
grid_result = grid.fit(X_train_scaled, y_train)

print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'Mean: {mean:.4f}, StdDev: {stdev:.4f}, Params: {param}')




### Explanation**

This code builds, trains, evaluates, and fine-tunes an Artificial Neural Network (ANN) to predict concrete strength based on input features. The workflow involves the following steps:

1. **Data Preparation**: 
   - The dataset is loaded, and duplicate rows are removed to ensure data quality. Features (`X`) and target (`y`) variables are separated, and the dataset is split into training and testing subsets (80-20 split). Feature scaling is applied using `StandardScaler` to standardize the input features, improving model performance and training stability.

2. **Model Definition**:
   - A sequential model is constructed with three hidden layers using the `relu` activation function for non-linear transformations. The final output layer uses a `linear` activation for regression. The model is compiled with the Adam optimizer, `mean_squared_error` as the loss function, and `mae` (Mean Absolute Error) as a performance metric.

3. **Model Training**:
   - The model is trained for 200 epochs with a batch size of 32. A validation split of 20% ensures performance is monitored on unseen data during training. The training history is stored for visualization.

4. **Evaluation and Predictions**:
   - The model's performance is evaluated on the test dataset using loss (`mean_squared_error`), MAE, and R² score to assess predictive accuracy. Predicted values (`y_pred`) are compared against actual test values.

5. **Performance Visualization**:
   - Training and validation losses, along with Mean Absolute Errors (MAE), are plotted across epochs to observe the model's convergence and detect signs of overfitting or underfitting.

6. **Hyperparameter Tuning**:
   - A dynamic model creation function is defined, allowing flexible specification of parameters like learning rate, dropout rate, activation functions, weight initializers, and the number of neurons in each layer. 
   - Using `GridSearchCV` with `KFold` cross-validation, the model is fine-tuned by testing multiple combinations of hyperparameters to find the optimal configuration. Results include the best model and detailed scores for all parameter combinations.

This comprehensive approach ensures robust model training, evaluation, and optimization, providing insights into both the model's strengths and potential areas for improvement.


### **Fashion MNIST Classification Project**


In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

class_labels = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

plt.figure(figsize=(16, 16))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(X_train[i], cmap="Greys")
    plt.axis('off')
    plt.title(f"{class_labels[y_train[i]]} ({y_train[i]})", fontsize=10)
plt.show()

X_train = X_train / 255.0
X_test = X_test / 255.0

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(units=32, activation='relu'),
    keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, verbose=1)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

y_pred = model.predict(X_test)

plt.figure(figsize=(16, 16))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(X_test[i], cmap="Greys")
    plt.axis('off')
    plt.title(f"Actual: {class_labels[y_test[i]]}\nPredicted: {class_labels[np.argmax(y_pred[i])]}")
plt.show()

cm = confusion_matrix(y_test, [np.argmax(pred) for pred in y_pred])
plt.figure(figsize=(16, 9))
sns.heatmap(cm, annot=True, fmt="d", cmap="coolwarm", xticklabels=class_labels, yticklabels=class_labels)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

cr = classification_report(y_test, [np.argmax(pred) for pred in y_pred], target_names=class_labels)
print("Classification Report:\n", cr)

model.save("MNIST_classifier_nn_model.h5")
loaded_model = keras.models.load_model("MNIST_classifier_nn_model.h5")

loaded_pred = loaded_model.predict(X_test)


### **Explanation**

This project demonstrates the use of a simple Artificial Neural Network (ANN) to classify images from the Fashion MNIST dataset. The dataset contains grayscale images of 28x28 pixels, each representing one of 10 fashion categories (e.g., T-shirt, dress, or sneakers). Below is the workflow:

1. **Importing Libraries**:
   - Essential libraries like Keras, NumPy, Matplotlib, and Seaborn are imported for model building, data manipulation, and visualization.
   
2. **Dataset Loading**:
   - The Fashion MNIST dataset is loaded, split into training (60,000 images) and testing (10,000 images) sets.
   - Labels are mapped to class names like T-shirt, Trouser, Dress, etc., for interpretability.

3. **Data Visualization**:
   - Sample images from the training set are visualized along with their labels to understand the data distribution and format.

4. **Feature Scaling**:
   - Pixel values are normalized to a range of 0 to 1 by dividing them by 255. This scaling improves the convergence of the neural network during training.

5. **Model Construction**:
   - A simple ANN is built using Keras Sequential API:
     - **Flatten Layer**: Converts the 2D image input into a 1D array.
     - **Hidden Layer**: A dense layer with 32 neurons and ReLU activation for learning non-linear patterns.
     - **Output Layer**: A dense layer with 10 neurons (one for each class) and a softmax activation function for multi-class classification.

6. **Model Compilation**:
   - The model uses:
     - **Adam Optimizer**: For adaptive learning.
     - **Sparse Categorical Crossentropy Loss**: For multi-class classification.
     - **Accuracy Metric**: To evaluate model performance.

7. **Model Training**:
   - The model is trained on the normalized training dataset for 10 epochs to optimize weights and minimize loss.

8. **Model Evaluation**:
   - The trained model is evaluated on the test dataset, and metrics like loss and accuracy are reported.

9. **Prediction and Visualization**:
   - Predictions are made on the test dataset, and the results are visualized by comparing actual labels with predicted labels for a subset of images.

10. **Confusion Matrix**:
    - A confusion matrix is generated and visualized as a heatmap to understand the model's performance across all classes.

11. **Classification Report**:
    - Precision, recall, and F1-score for each class are displayed to provide a detailed analysis of the model's strengths and weaknesses.

12. **Model Saving and Reloading**:
    - The trained model is saved as a `.h5` file for future use and reloaded to verify its predictions on the test data.

This project highlights the key steps in building and deploying a neural network for image classification tasks while providing detailed performance insights through evaluation metrics.
